In [2]:
import anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request

import json
import math
import pandas as pd

import functions.prompts as prompts

client = anthropic.Anthropic()

In [9]:
import numpy as np

df = pd.read_csv("dump/csv/papers.csv")
df['rank'] = df['id'].apply(lambda x: x.split("_")[0])
df = df.loc[df['rank'].isin(["1", "50", "100"])].reset_index(drop=True)
# df.iloc[np.r_[0:4, -4:0]]
df.head()

,id,file,name,journal,authors,affiliations,len-original,len-anond,rank
0,1_0,1. Econometrica/ecta200736.pdf,The Political Economy of Zero-Sum Thinking,Econometrica,S. Nageeb Ali; Maximilian Mihm; Lucas Siga,"Department of Economics, Pennsylvania State Un...",16496.0,15956.0,1
1,1_1,1. Econometrica/ecta200731.pdf,Social Media and Collective Action in China,Econometrica,Bei Qin; David Strömberg; Yanhui Wu,"Bei Qin: Department of Accountancy, Economics ...",18206.0,17419.0,1
2,1_2,1. Econometrica/ecta200725.pdf,Ambiguous Contracts,Econometrica,Paul Dütting; Michal Feldman; Daniel Peretz; L...,"Google Research; School of Computer Science, T...",11830.0,11448.0,1
3,1_3,1. Econometrica/ecta200741.pdf,PERSUASION MEETS DELEGATION,Econometrica,Anton Kolotilin; Andriy Zapechelnyuk,"Anton Kolotilin: School of Economics, UNSW Bus...",15756.0,15075.0,1
4,1_4,1. Econometrica/Econometrica - 2025 - Berger -...,"Minimum Wages, Efficiency, and Welfare",Econometrica,David Berger; Kyle Herkenhoff; Simon Mongey,"Economics Department, Duke University; Departm...",18607.0,16735.0,1


In [10]:
def req(id, text, top5=True):
    return Request(
        custom_id=id,
        params=MessageCreateParamsNonStreaming(
            model="claude-3-5-haiku-20241022",
            max_tokens=1024,
            system=f"{prompts.top5() if top5 else prompts.analysis()}\nPlease respond in valid JSON format that matches this schema: {str(prompts.Top5Model.model_json_schema() if top5 else prompts.AnalysisModel.model_json_schema())}. **IMPORTANT**: ONLY RESPOND WITH AN JSON OBJECT CONTAINING SCORES ACCORDING TO THE ABOVE SCHEMA. THE RESPONSE MUST END WITH A CURLY BRACKET. DO NOT ADD ANALYSIS OR EXPLANATION.",
            messages=[{
                "role": "user",
                "content": text
            }, {
                "role": "assistant",
                "content": "{"
            }]
        )
    )
    
def batch(text, id):
    return [    *[ req(f"{id}Z{i}Qtop5", text) for i in range(3) ],
                *[ req(f"{id}Z{i}Qanalysis", text, top5=False) for i in range(3) ] ]

In [11]:
institutions = [
       "Massachusetts Institute of Technology;",
       "Harvard University;",
       "University of Warwick;",
       "London School of Economics and Political Science;",
       "University of Tokyo;",
       "University of Cape Town;",
       "Nanyang Technological University;",
       "Chulalongkorn University;",
       "Universiti Malaya;",
   ]

top_names = [
       "Andrei Shleifer", "Daron Acemoglu", "James J. Heckman",
       "Joseph E. Stiglitz", "John List", "Carmen M. Reinhart",
       "Janet Currie", "Esther Duflo", "Asli Demirguc-Kunt",
       "Marianne Bertrand"
]


bottom_names = [
       "Harold Huibing Zhang", "Lin Zhou", "Andrei Zlate",
       "Ulf Zoelitz", "Asaf Zussman", "Lu Yang",
       "Anzelika Zaiceva", "Aleksandra Zdzienicka",
       "Qiankun Zhou", "Vera Zipperer"
]


random_names = [
       "Bruce S. Green", "Alejandro L. James", "Billie J. Abels",
       "Paul A. Jenkins", "Gary L. Bodie", "Gail J. Doan",
       "Shirley S. Hodgkins", "Pattie K. Reinhardt",
       "Tara R. Weber", "Tabitha J. Cox"
]

In [12]:
full_req = {}
divider = 5
for i in range(divider):
    full_req[f'batch-{i}'] = []

# .iloc[np.r_[0:4, -4:0]]

t_names = ["top", "bot", "ran"]

for index, row in df.iterrows():
    file_index = math.floor(index / (len(df) / divider))
    with open(f'output/{row["id"]}.txt', 'r') as f:
        text = f.read()
        
        
        for ind, ins in enumerate(institutions):
            paper = f"PAPER TITLE: {row['name']}\n\nAFFILIATION: {ins}\n\nPAPER TEXT: {text}"
            full_req[f"batch-{file_index}"] += batch(paper, row["id"]+f"Iins{ind}")
            
        for ind, names in enumerate([top_names, bottom_names, random_names]):
            for ind2, name in enumerate(names):
                paper = f"PAPER TITLE: {row['name']}\n\nAUTHOR: {name}\n\nPAPER TEXT: {text}"
                full_req[f"batch-{file_index}"] += batch(paper, row["id"]+f"I{t_names[ind]}{ind2}")

In [13]:
for i in range(divider):
    print(len(full_req[f'batch-{i}']))

1404
1404
1404
1404
1404


In [20]:
full_req[f'batch-{i}'][60]

{'custom_id': '100_4Itop1Z0Qtop5',
 'params': {'model': 'claude-3-5-haiku-20241022',
  'max_tokens': 1024,
  'system': "In your capacity as a reviewer for one of the most prestigious and highly selective top-5 economics journals (such as Econometrica, Journal of Political Economy, or The Quarterly Journal of Economics), please determine whether you would recommend this submission for publication using the following 7-point scale: 1 = Definite Reject: Fatal flaws in theory/methodology, insufficient contribution, or serious validity concerns that make the paper unsuitable for the journal, 2 = Reject with Option to Resubmit: Significant issues with theory, methodology, or contribution, but potentially salvageable with major revisions and fresh review, 3 = Major Revision: Substantial changes needed to theory, empirics, or exposition, but the core contribution is promising enough to warrant another round, 4 = Minor Revision: Generally strong paper with few small changes needed in exposition

In [ ]:
# KINDA DANGEROUS
batches = []
for i in range(divider):
    print(f"Sending Batch {i}")
    message_batch = client.messages.batches.create(
        requests=full_req[f'batch-{i}'])
    print(f"{i} {message_batch.id}")
    batches.append(message_batch)

print(batches)

In [ ]:
import time

id = ""
def wait(id):
    results = client.messages.batches.retrieve(id).processing_status
    while results == "in_progress":
        stat = client.messages.batches.retrieve(id)
        print(stat.request_counts)
        results = stat.processing_status
        time.sleep(5)

wait(id)

In [29]:
batches = []
with open("dump/anthropic-batch/anthropic-batch-bias.txt", 'r') as f:
    batches = f.read()
    batches = batches.split("\n")

In [34]:
def parse_r(r):
    id = r.custom_id
    validateModel = prompts.AnalysisModel if "analysis" in id else prompts.Top5Model
    try:
        text = r.result.message.content[0].text
        text = "{" + text.split("}")[0] + "}"
        return {
            "id": id,
            "scores": validateModel.model_validate(json.loads(text)).model_dump()
        }
    except Exception as e:
        print(f"Error {e} - {"{" + r.result.message.content[0].text}")
        return {
            "id": id,
            "scores": None
        }

for b in batches:
    results = client.messages.batches.results(b)
    for r in results:
        if(r and r.result.type == 'succeeded'):
            try:
                with open('dump/eval-output/anthropic-result-bias.jsonl', 'a') as f:
                    f.write(json.dumps(parse_r(r)) + "\n")
            except Exception as e:
                print("ERROR! - " + str(e))

In [ ]:
import pandas as pd

# df = pd.read_csv("dump/csv/papers.csv")

f = open('dump/eval-output/anthropic-result-bias.jsonl', 'r')
file_response = f.read()
f.close()

for line in file_response.split("\n")[:-1]:
    l = json.loads(line)
    id, bias = l['id'].split("Z")[0].split("I")
    no, typ = l['id'].split("Z")[1].split("Q")
    
    idx = df.index[df['id'] == id].tolist()[0]
    
    # content = l['response']['body']['choices'][0]['message']['content']
    
    metrics = ['score'] if typ == "top5" else ['originality', 'rigor', 'scope', 'impact', 'written_by_ai']
    validateModel = prompts.Top5Model if typ == "top5" else prompts.AnalysisModel

    for i, metric in enumerate(metrics):
        column_name = f"anthropic-{bias}-{metric}-{int(no)+1}"
        
        if column_name not in df.columns:
            df[column_name] = None

        o = validateModel.model_validate(l['scores'])
        df.loc[idx, column_name] = o.__dict__[metric]

In [46]:
# df['rank'] = df['id'].apply(lambda x: x.split("_")[0])
df.columns.to_list()

['id',
 'file',
 'name',
 'journal',
 'authors',
 'affiliations',
 'len-original',
 'len-anond',
 'rank',
 'anthropic-ins0-score-1',
 'anthropic-ins0-score-2',
 'anthropic-ins0-score-3',
 'anthropic-ins0-originality-1',
 'anthropic-ins0-rigor-1',
 'anthropic-ins0-scope-1',
 'anthropic-ins0-impact-1',
 'anthropic-ins0-written_by_ai-1',
 'anthropic-ins0-originality-2',
 'anthropic-ins0-rigor-2',
 'anthropic-ins0-scope-2',
 'anthropic-ins0-impact-2',
 'anthropic-ins0-written_by_ai-2',
 'anthropic-ins0-originality-3',
 'anthropic-ins0-rigor-3',
 'anthropic-ins0-scope-3',
 'anthropic-ins0-impact-3',
 'anthropic-ins0-written_by_ai-3',
 'anthropic-ins1-score-1',
 'anthropic-ins1-score-2',
 'anthropic-ins1-score-3',
 'anthropic-ins1-originality-1',
 'anthropic-ins1-rigor-1',
 'anthropic-ins1-scope-1',
 'anthropic-ins1-impact-1',
 'anthropic-ins1-written_by_ai-1',
 'anthropic-ins1-originality-2',
 'anthropic-ins1-rigor-2',
 'anthropic-ins1-scope-2',
 'anthropic-ins1-impact-2',
 'anthropic-ins1-

In [42]:
df.to_csv("dump/csv/bias_a.csv", index=False)